# Question Answering
We discussed Document Loading and Splitting as well as Storage and Retrieval.

Let's load our vectorDB.

In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

The code below was added to assign the openai LLM version filmed until it is deprecated, currently in Sept 2023. LLM responses can often vary, but the responses may be significantly different when using a different model version.

In [2]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo-0301


In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [4]:
print(vectordb._collection.count())

149


In [5]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [6]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

## RetrievalQA chain

In [7]:
from langchain.chains import RetrievalQA

In [8]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [9]:
result = qa_chain({"query": question})

In [10]:
result["result"]

'The class being referred to is not specified in the given context. However, based on the provided context, some major topics covered in the CS229 lecture notes by Andrew Ng include generative learning algorithms, discriminative learning algorithms, supervised learning, and support vector machines.'

## Prompt

In [11]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [12]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [13]:
question = "Is probability a class topic?"

In [14]:
result = qa_chain({"query": question})

In [15]:
result["result"]

'Yes, probability is a class topic. Thanks for asking!'

In [16]:
result["source_documents"][0]

Document(page_content='ingto\x1eyornot,sincewewilltypically haveafairfraction eachofspam and\nnon-spam messages, so\x1eywillbeareasonable estimate ofp(y=1)andwill\nbequite farfrom0anyway.)\n2.2Eventmodelsfortextclassi\x0ccation\nTocloseo\x0bourdiscussion ofgenerativ elearning algorithms, letstalkabout\nonemoremodelthatisspeci\x0ccally fortextclassi\x0ccation. While NaiveBayes', metadata={'page': 11, 'source': 'C:\\Users\\alexm\\AppData\\Local\\Temp\\tmpgsbgz6ge\\tmp.pdf'})

## RetrievalQA chain types¶

In [17]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [ ]:
result = qa_chain_mr({"query": question})

In [18]:
result["result"]

'Yes, probability is a class topic. Thanks for asking!'

If you wish to experiment on the LangChain plus platform:

Go to langchain plus platform and sign up
Create an API key from your account's settings
Use this API key in the code below
uncomment the code
Note, the endpoint in the video differs from the one below. Use the one below.